## 下载环境

pip install transformers
pip install modelscope
pip install datasets
pip install accelerate
pip install bitsandbytes 
pip install peft
pip install swanlab
pip install sentencepiece
pip install trl 


pip install unsloth

In [1]:
!pip list

Package                        Version
------------------------------ ---------------
absl-py                        2.0.0
accelerate                     1.7.0
addict                         2.4.0
aiohappyeyeballs               2.6.1
aiohttp                        3.11.18
aiosignal                      1.3.2
annotated-types                0.7.0
anyio                          4.2.0
argon2-cffi                    23.1.0
argon2-cffi-bindings           21.2.0
arrow                          1.3.0
asttokens                      2.4.1
async-lru                      2.0.4
async-timeout                  5.0.1
attrs                          23.2.0
Babel                          2.14.0
beautifulsoup4                 4.12.2
bitsandbytes                   0.45.5
bleach                         6.1.0
boto3                          1.38.20
botocore                       1.38.20
brotlipy                       0.7.0
cachetools                     5.3.2
certifi                        2022.12.7
cffi      

## 下载数据集

In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset


ds_reason = load_dataset("Ronndy/medical_o1_sft_Chinese",cache_dir='./data/reason')
ds_no_reason = load_dataset("BAAI/IndustryInstruction_Health-Medicine",cache_dir = './data/no_reason')

medical_o1_sft_Chinese_train_v1.json:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

medical_o1_sft_Chinese_test_v1.json:   0%|          | 0.00/10.9M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
ds_reason

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 17340
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 3717
    })
})

In [3]:
#数据集保存
#推理数据保存
ds_reason.save_to_disk("./local_reason_dataset")
#非推理数据保存
ds_no_reason.save_to_disk("./local_no_reason_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/17340 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3717 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/348015 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
#数据集从 本地 加载
from datasets import load_from_disk,DatasetDict, Dataset

data_reason = load_from_disk("./local_reason_dataset")
data_reason = DatasetDict({
    'train': Dataset.from_dict(data_reason['train'][:1600]) 
})
data_no_reason = load_from_disk("./local_no_reason_dataset")
data_no_reason = DatasetDict({
    'train': Dataset.from_dict(data_no_reason['train'][:400]) 
})

In [11]:
data_reason,data_no_reason

(DatasetDict({
     train: Dataset({
         features: ['messages'],
         num_rows: 1600
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['instruction', 'id', 'conversations', 'lang'],
         num_rows: 400
     })
 }))

In [12]:
data_reason['train'][0]

{'messages': [{'content': '你是一个医生，能够根据病人的表述为病人诊断病情，当然你也有跟人交流普通问题的情况。',
   'role': 'system'},
  {'content': '一刚出生新生儿,身体红,四肢青紫,心率90次/分,呼吸20次/分,不规则,四肢略屈曲,弹足底有皱眉。该新生儿Apgar评分为\nA. 8分\nB. 7分\nC. 6分\nD. 5分\nE. 4分',
   'role': 'user'},
  {'content': '<think>\n嗯，这个新生儿刚出生身体是红色的，但是四肢是青紫色的。看上去身体的血液循环似乎还不错，但是四肢稍微差一点。躯干部位在Apgar评分中可以算2分，四肢青紫得不了分，所以颜色这部分我认为总共可以算1分。\n\n哦，他的心率是90次每分钟。根据Apgar评分的标准，心率要超过100才可以得2分。90次不到100次但还不是最糟糕的情况下，可以算1分。\n\n然后是呼吸情况，宝宝的呼吸好像有点不规则。对于Apgar评分来说，不规则呼吸大概算是1分吧，而正常呼吸是2分的标准。这样看来，呼吸部分只得1分。\n\n肌张力方面，小家伙四肢略微屈曲。完全屈曲是最好的，得2分，但他只是稍微屈曲，所以这里给1分妥当。\n\n哦对，反射兴奋性这个也要看一下。他在弹足底时有皱眉的反应。这个皱眉呢，在评分中算是轻度反应，得1分。要是能有更明显的反应比如哭，那才能得2分。\n\n综合来看，这个小宝宝在评分上颜色1分，心率1分，呼吸1分，肌张力1分，反射兴奋性1分，总共加起来应该是5分。果然是D，没错。\n</think>\n<response>根据Apgar评分标准：\n\n1. 颜色：身体红、四肢青紫，计1分。\n2. 心率：90次/分，计1分。\n3. 呼吸：不规则，计1分。\n4. 肌张力：四肢略屈曲，计1分。\n5. 反射兴奋性：弹足底有皱眉，计1分。\n\n综合以上五项指标，该新生儿的Apgar评分为5分。因此，正确选项是D. 5分。</response>',
   'role': 'assistant'}]}

In [13]:
data_no_reason['train'][0]

{'instruction': '',
 'id': 'industry-instruction-edu',
 'conversations': [{'from': 'human',
   'value': '问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关'},
  {'from': 'gpt',
   'value': '答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。'},
  {'from': 'human', 'value': '问：现在这个囊肿就已经压迫脑干了，如果以后再长大把脑干都压上了以后手术是不是就很难很危险了'},
  {'from': 'gpt',
   'value': '答：如果囊肿已经压迫了脑干，那么情况确实比较复杂和危险。手术可能会变得更具风险，因为脑干是一个重要的神经结构，手术过程中需要小心以避免额外的损伤。\n\n不过，你的具体情况需要医生进行详细的评估和决策。专业医生将会考虑多个因素，包括囊肿的位置、大小、对周围结构的影响以及你的整体健康状况等等。他们会权衡手术的利弊，并与你详细讨论手术风险和可能的好处。\n\n最重要的是，请务必在专业医生的指导下进行进一步的治疗和管理。他们能够为你提供最佳的个性化建议，并指导你选择最适合你情况的治疗方案。'}],
 'lang': 'zh'}

In [7]:
type(data_reason['train'])

datasets.arrow_dataset.Dataset

In [14]:
type(data_no_reason['train'])

datasets.arrow_dataset.Dataset

#### sharegpt 格式数据集处理

In [5]:
from unsloth.chat_templates import standardize_sharegpt
data_no_reason = standardize_sharegpt(data_no_reason['train'])
data_no_reason[0]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2025-05-22 23:49:54,523] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

🦥 Unsloth Zoo will now patch everything to make training faster!


Unsloth: Standardizing formats (num_proc=128):   0%|          | 0/400 [00:00<?, ? examples/s]

{'instruction': '',
 'id': 'industry-instruction-edu',
 'conversations': [{'content': '问：右侧桥小区囊性占位，囊肿可能大。偏肢体发育不良，巨脑回；右侧桥小脑区囊性占位，囊肿可能大.曾经治疗情况和效果：左手左脚没有右面灵活，爱往左歪脖，曾经感冒发烧39度抽过3、4次，抽时症状：身体僵硬呕吐。不知道是和囊肿有关还是和发育不良有关',
   'role': 'user'},
  {'content': '答：根据你描述的症状和病史，右侧桥小区囊性占位可能是导致偏肢体发育不良和巨脑回的原因之一。囊肿的大小可能也会对症状产生影响。\n\n至于感冒发烧抽搐的症状，这可能与囊肿有关，也可能与发育不良有关。囊肿对中枢神经系统的正常功能可能会有影响，导致肌肉僵硬和呕吐等症状。\n\n然而，请注意，我只是凭借你提供的信息做出初步推断，而不能提供确切的诊断。为了明确病因和确定最佳治疗方案，我建议你尽快咨询专业的医生进行详细的检查和进一步的评估。他们可以根据你的具体情况给出准确的诊断和建议。',
   'role': 'assistant'},
  {'content': '问：现在这个囊肿就已经压迫脑干了，如果以后再长大把脑干都压上了以后手术是不是就很难很危险了', 'role': 'user'},
  {'content': '答：如果囊肿已经压迫了脑干，那么情况确实比较复杂和危险。手术可能会变得更具风险，因为脑干是一个重要的神经结构，手术过程中需要小心以避免额外的损伤。\n\n不过，你的具体情况需要医生进行详细的评估和决策。专业医生将会考虑多个因素，包括囊肿的位置、大小、对周围结构的影响以及你的整体健康状况等等。他们会权衡手术的利弊，并与你详细讨论手术风险和可能的好处。\n\n最重要的是，请务必在专业医生的指导下进行进一步的治疗和管理。他们能够为你提供最佳的个性化建议，并指导你选择最适合你情况的治疗方案。',
   'role': 'assistant'}],
 'lang': 'zh'}

### aipaca数据集处理

In [29]:
def convert_to_alpaca_format(examples):
    conversations = []
    for messages in examples["messages"]:
        # 提取系统提示词
        system_prompt = next((msg["content"] for msg in messages if msg["role"] == "system"), "") #next(..., "")	安全取值	返回首个匹配项或空字符串默认值
        
        # 提取用户问题和助手回答
        user_question = next((msg["content"] for msg in messages if msg["role"] == "user"), "")
        assistant_answer = next((msg["content"] for msg in messages if msg["role"] == "assistant"), "")
        
        # 分割think和response部分
        think_part = ""
        response_part = ""
        if assistant_answer:
            think_start = assistant_answer.find("<think>")
            think_end = assistant_answer.find("</think>")
            if think_start != -1 and think_end != -1:
                think_part = assistant_answer[think_start+7:think_end].strip()
            
            response_start = assistant_answer.find("<response>")
            response_end = assistant_answer.find("</response>")
            if response_start != -1 and response_end != -1:
                response_part = assistant_answer[response_start+10:response_end].strip()
            else:
                # 如果没有response标签，尝试获取think标签之后的内容
                response_part = assistant_answer[think_end+8:].strip()
        
        # 构建Alpaca格式的对话
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question},
            {"role": "assistant", "content": f'<think>{think_part}</think>{response_part}'}
        ]
        conversations.append(conversation)
    
    return {"conversations": conversations}

# 批量处理数据集

data_reason_alpaca = data_reason['train'].map(
    convert_to_alpaca_format,
    batched=True, #默认一个批次大小为1000
    remove_columns=data_reason["train"].column_names,
)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [30]:
data_reason_alpaca[0]

{'conversations': [{'content': '你是一个医生，能够根据病人的表述为病人诊断病情，当然你也有跟人交流普通问题的情况。',
   'role': 'system'},
  {'content': '一刚出生新生儿,身体红,四肢青紫,心率90次/分,呼吸20次/分,不规则,四肢略屈曲,弹足底有皱眉。该新生儿Apgar评分为\nA. 8分\nB. 7分\nC. 6分\nD. 5分\nE. 4分',
   'role': 'user'},
  {'content': '<think>嗯，这个新生儿刚出生身体是红色的，但是四肢是青紫色的。看上去身体的血液循环似乎还不错，但是四肢稍微差一点。躯干部位在Apgar评分中可以算2分，四肢青紫得不了分，所以颜色这部分我认为总共可以算1分。\n\n哦，他的心率是90次每分钟。根据Apgar评分的标准，心率要超过100才可以得2分。90次不到100次但还不是最糟糕的情况下，可以算1分。\n\n然后是呼吸情况，宝宝的呼吸好像有点不规则。对于Apgar评分来说，不规则呼吸大概算是1分吧，而正常呼吸是2分的标准。这样看来，呼吸部分只得1分。\n\n肌张力方面，小家伙四肢略微屈曲。完全屈曲是最好的，得2分，但他只是稍微屈曲，所以这里给1分妥当。\n\n哦对，反射兴奋性这个也要看一下。他在弹足底时有皱眉的反应。这个皱眉呢，在评分中算是轻度反应，得1分。要是能有更明显的反应比如哭，那才能得2分。\n\n综合来看，这个小宝宝在评分上颜色1分，心率1分，呼吸1分，肌张力1分，反射兴奋性1分，总共加起来应该是5分。果然是D，没错。</think>根据Apgar评分标准：\n\n1. 颜色：身体红、四肢青紫，计1分。\n2. 心率：90次/分，计1分。\n3. 呼吸：不规则，计1分。\n4. 肌张力：四肢略屈曲，计1分。\n5. 反射兴奋性：弹足底有皱眉，计1分。\n\n综合以上五项指标，该新生儿的Apgar评分为5分。因此，正确选项是D. 5分。',
   'role': 'assistant'}]}

In [31]:
from unsloth import FastLanguageModel
model_name = "/root/autodl-tmp/Qwen/Qwen3-4B"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2025-05-22 23:34:44,486] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

🦥 Unsloth Zoo will now patch everything to make training faster!


In [32]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,   
        
    full_finetuning = False,    
)

==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.57 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
# 将转换后的推理数据集应用对话模板
reasoning_conversations = tokenizer.apply_chat_template(
    data_reason_alpaca["conversations"],
    tokenize = False
)

In [34]:
reasoning_conversations[0]

'<|im_start|>system\n你是一个医生，能够根据病人的表述为病人诊断病情，当然你也有跟人交流普通问题的情况。<|im_end|>\n<|im_start|>user\n一刚出生新生儿,身体红,四肢青紫,心率90次/分,呼吸20次/分,不规则,四肢略屈曲,弹足底有皱眉。该新生儿Apgar评分为\nA. 8分\nB. 7分\nC. 6分\nD. 5分\nE. 4分<|im_end|>\n<|im_start|>assistant\n<think>\n嗯，这个新生儿刚出生身体是红色的，但是四肢是青紫色的。看上去身体的血液循环似乎还不错，但是四肢稍微差一点。躯干部位在Apgar评分中可以算2分，四肢青紫得不了分，所以颜色这部分我认为总共可以算1分。\n\n哦，他的心率是90次每分钟。根据Apgar评分的标准，心率要超过100才可以得2分。90次不到100次但还不是最糟糕的情况下，可以算1分。\n\n然后是呼吸情况，宝宝的呼吸好像有点不规则。对于Apgar评分来说，不规则呼吸大概算是1分吧，而正常呼吸是2分的标准。这样看来，呼吸部分只得1分。\n\n肌张力方面，小家伙四肢略微屈曲。完全屈曲是最好的，得2分，但他只是稍微屈曲，所以这里给1分妥当。\n\n哦对，反射兴奋性这个也要看一下。他在弹足底时有皱眉的反应。这个皱眉呢，在评分中算是轻度反应，得1分。要是能有更明显的反应比如哭，那才能得2分。\n\n综合来看，这个小宝宝在评分上颜色1分，心率1分，呼吸1分，肌张力1分，反射兴奋性1分，总共加起来应该是5分。果然是D，没错。\n</think>\n\n根据Apgar评分标准：\n\n1. 颜色：身体红、四肢青紫，计1分。\n2. 心率：90次/分，计1分。\n3. 呼吸：不规则，计1分。\n4. 肌张力：四肢略屈曲，计1分。\n5. 反射兴奋性：弹足底有皱眉，计1分。\n\n综合以上五项指标，该新生儿的Apgar评分为5分。因此，正确选项是D. 5分。<|im_end|>\n'

## 注意事项